# Core10_11 — Governance Stress Test Suite

## 목적
Core10 운영 모드가 입력 교란(stress) 하에서도
- 결정론을 유지하는지
- 정책 제약을 위반하지 않는지
- 운영 지표가 붕괴하지 않는지
를 검증한다.

## Stress Types
1. SoMS 관측 노이즈
2. 센서 누락 / drop
3. Toggle spike (급격한 진동)

## 출력
- stress_test_result_table
  - pass / fail
  - failure reason
  - 주요 지표 변화량

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path

# 입력: core10_07 결과
ART_CORE10 = Path("../artifact/core10")
ROLLOUT_PATH = ART_CORE10 / "core10_07_operation_rollout_log.csv"

assert ROLLOUT_PATH.exists(), ROLLOUT_PATH.resolve()
rollout_base = pd.read_csv(ROLLOUT_PATH)

print("Loaded rollout:", rollout_base.shape)
rollout_base.head(3)

Loaded rollout: (60, 22)


,run_id,case_id,step,selected_antibody_id,selected_signature,allocation_id,allocation_changed,proxy_survivability_score,tie_break_risk,core10_operational_risk,...,want_switch,switched,switch_reason,cooldown_active,ttl_active,last_switch_step,ttl_until_step,SoMS,toggle_rate,drift
0,core10_sim_001,B_GOVERNED,0,GDPa1-060,IgG1|Kappa|0,GDPa1-060,0,0.858995,1.308744,0.156673,...,0,0,NO_SWITCH,1,1,0,8,0.181387,0.0,0.015
1,core10_sim_001,B_GOVERNED,1,GDPa1-060,IgG1|Kappa|0,GDPa1-060,0,0.858995,1.308744,0.156673,...,0,0,NO_SWITCH,1,1,0,8,0.376939,0.0,0.030
2,core10_sim_001,B_GOVERNED,2,GDPa1-060,IgG1|Kappa|0,GDPa1-060,0,0.858995,1.308744,0.156673,...,0,0,NO_SWITCH,1,1,0,8,0.523711,0.0,0.045


## Stress Test 판단 기준 (고정)

각 stress scenario에서 아래 조건을 검사한다.

### PASS 조건
- 정책 위반 없음 (cooldown/TTL 무시 없음)
- SoMS slope 폭증하지 않음
- 재설계 요청이 "즉시 무한 루프"로 가지 않음

### FAIL 조건
- redesign_request_time이 비정상적으로 0~2 step
- soms_degradation_rate가 baseline 대비 X배 이상
- toggle_rate가 saturation 후 복구 불가

In [7]:
def redesign_request_time(df, soms_threshold=15.0):
    """
    First step where SoMS exceeds redesign threshold.
    If never exceeded, return max step.
    """
    hit = df[df["SoMS"] >= soms_threshold]
    if hit.empty:
        return int(df["step"].max())
    return int(hit["step"].iloc[0])


def soms_degradation_rate(df):
    """
    Linear slope of SoMS over time (ΔSoMS / step)
    """
    if df["step"].nunique() < 2:
        return 0.0
    coef = np.polyfit(df["step"], df["SoMS"], 1)
    return float(coef[0])


def redesign_request_frequency(df, soms_threshold=15.0):
    """
    Count how often SoMS crosses threshold from below.
    """
    s = df["SoMS"] >= soms_threshold
    crossings = (s.astype(int).diff() == 1).sum()
    return int(crossings)

In [8]:
baseline_metrics = {
    "redesign_request_time": redesign_request_time(rollout_base),
    "soms_degradation_rate": soms_degradation_rate(rollout_base),
    "redesign_request_frequency": redesign_request_frequency(rollout_base),
}

baseline_metrics

{'redesign_request_time': 47,
 'soms_degradation_rate': 0.3641330196493166,
 'redesign_request_frequency': 1}

In [9]:
def inject_soms_noise(df, sigma=0.5, seed=42):
    rng = np.random.default_rng(seed)
    out = df.copy()
    noise = rng.normal(0, sigma, size=len(out))
    out["SoMS"] = np.maximum(0.0, out["SoMS"] + noise)
    out["stress_tag"] = f"SoMS_noise_sigma_{sigma}"
    return out


def inject_sensor_drop(df, drop_rate=0.2, seed=42):
    rng = np.random.default_rng(seed)
    out = df.copy()
    mask = rng.random(len(out)) < drop_rate
    out.loc[mask, "SoMS"] = np.nan
    out["SoMS"] = out["SoMS"].ffill().fillna(0.0)
    out["stress_tag"] = f"sensor_drop_{drop_rate}"
    return out


def inject_toggle_spike(df, spike_step=30, spike_value=1.5):
    out = df.copy()
    out.loc[out["step"] == spike_step, "toggle_rate"] = spike_value
    out["stress_tag"] = f"toggle_spike_step_{spike_step}"
    return out

In [10]:
stress_scenarios = [
    {
        "stress_key": "SoMS_noise_low",
        "fn": lambda df: inject_soms_noise(df, sigma=0.3),
    },
    {
        "stress_key": "SoMS_noise_high",
        "fn": lambda df: inject_soms_noise(df, sigma=1.0),
    },
    {
        "stress_key": "sensor_drop_20pct",
        "fn": lambda df: inject_sensor_drop(df, drop_rate=0.2),
    },
    {
        "stress_key": "sensor_drop_40pct",
        "fn": lambda df: inject_sensor_drop(df, drop_rate=0.4),
    },
    {
        "stress_key": "toggle_spike",
        "fn": lambda df: inject_toggle_spike(df, spike_step=30),
    },
]

In [11]:
rows = []

for sc in stress_scenarios:
    stressed = sc["fn"](rollout_base)

    metrics = {
        "redesign_request_time": redesign_request_time(stressed),
        "soms_degradation_rate": soms_degradation_rate(stressed),
        "redesign_request_frequency": redesign_request_frequency(stressed),
    }

    # PASS / FAIL rules
    fail_reasons = []

    if metrics["redesign_request_time"] <= 2:
        fail_reasons.append("early_redesign_trigger")

    if metrics["soms_degradation_rate"] > baseline_metrics["soms_degradation_rate"] * 2.0:
        fail_reasons.append("soms_slope_explosion")

    if stressed["toggle_rate"].max() >= 1.5 and stressed["toggle_rate"].iloc[-1] >= 1.4:
        fail_reasons.append("toggle_no_recovery")

    rows.append({
        "stress_key": sc["stress_key"],
        "pass": len(fail_reasons) == 0,
        "fail_reason": ",".join(fail_reasons) if fail_reasons else "",
        **metrics,
    })

stress_results = pd.DataFrame(rows)
stress_results

,stress_key,pass,fail_reason,redesign_request_time,soms_degradation_rate,redesign_request_frequency
0,SoMS_noise_low,False,toggle_no_recovery,46,0.365174,1
1,SoMS_noise_high,False,toggle_no_recovery,46,0.364850,1
2,sensor_drop_20pct,False,toggle_no_recovery,47,0.358152,1
3,sensor_drop_40pct,False,toggle_no_recovery,48,0.356608,1
4,toggle_spike,False,toggle_no_recovery,47,0.364133,1


## 결과 해석 가이드

- pass = True
  → 운영 정책이 해당 교란에 대해 안정적

- early_redesign_trigger
  → 작은 관측 오차에도 즉시 재설계 요청 → 과민

- soms_slope_explosion
  → 정책이 stress를 흡수하지 못함

- toggle_no_recovery
  → oscillation dampening 실패

In [12]:
OUT_PATH = ART_CORE10 / "core10_11_governance_stress_robustness.csv"
stress_results.to_csv(OUT_PATH, index=False)

print("✅ Exported stress test results:")
print(OUT_PATH.resolve())
stress_results

✅ Exported stress test results:
/Users/mac/Desktop/De/Developability_Data/core/artifact/core10/core10_11_governance_stress_robustness.csv


,stress_key,pass,fail_reason,redesign_request_time,soms_degradation_rate,redesign_request_frequency
0,SoMS_noise_low,False,toggle_no_recovery,46,0.365174,1
1,SoMS_noise_high,False,toggle_no_recovery,46,0.364850,1
2,sensor_drop_20pct,False,toggle_no_recovery,47,0.358152,1
3,sensor_drop_40pct,False,toggle_no_recovery,48,0.356608,1
4,toggle_spike,False,toggle_no_recovery,47,0.364133,1
